In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Dec 10 11:25:55 2024

@author: kamand
"""
import cv2
import numpy
import numpy as np
import pickle
from tensorflow.keras.models import load_model
import sys
from PyQt5.QtWidgets import (
    QApplication,
    QMainWindow,
    QLabel,
    QPushButton,
    QVBoxLayout,
    QHBoxLayout,
    QFileDialog,
    QWidget,
    QButtonGroup,
)
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QMessageBox
file_path = 0;

class GetImageAndAddModule(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Add your patient's Image")
        self.setGeometry(100, 100, 500, 500)
        self.setStyleSheet("background-color: black; color: white")
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout()
        self.images = QHBoxLayout()
        
        
        # title of loading
        self.image_label = QLabel("Load an image of your patient")
   #     self.preprocessed_label1 = QLabel("pre images 1")
     #   self.preprocessed_label2 = QLabel("pre image 2")
      #  self.preprocessed_label3 = QLabel("pre image 3")
        
        
        self.image_label.setStyleSheet("""
                                       background-color: #1b2b2b; 
                                       color : white;
                                       border: 2px solid white; 
                                       """
            ) 
        self.image_label.setAlignment(Qt.AlignCenter) 
        self.image_label.mousePressEvent = self.clicked_thetopmessage
        self.layout.addWidget(self.image_label)

        # Here I'm adding button
        
        
        self.process_button = QPushButton("PreProcessing")
        self.process_button.setStyleSheet("""
                                             QPushButton{
                                                 
                                                 background-color: #4CAF50; 
                                                 color: white; 
                                                 font-weight : bold;
                                                 border = 2px solid white;
                                                 padding = 5px;
                              }
                            """
            )     
        self.process_button.clicked.connect(self.processing)
        self.layout.addWidget(self.process_button)
        
        
        self.load_image_button = QPushButton("Load Image")
        self.load_image_button.setStyleSheet("""
                                             QPushButton{
                                                 
                                                 background-color: #4CAF50; 
                                                 color: white; 
                                                 font-weight : bold;
                                                 border = 2px solid white;
                                                 padding = 5px;
                              }
                            """
            )
        
        self.load_image_button.clicked.connect(self.load_image)
        self.layout.addWidget(self.load_image_button)

        
        self.button1 = QPushButton("SVM")
        self.button1.setStyleSheet("""
                                   QPushButton {
                                       background-color: #2b2b2b;
                                       color: white;
                                       font-weight = bold;
                                       border : 2px sold white;
                                       padding : 5px;
                
                    }
                    """
            )
        self.button1.clicked.connect(self.SVM)
        self.layout.addWidget(self.button1)



        self.button2 = QPushButton("K-Means")
        self.button2.setStyleSheet("""
                                   QPushButton {
                                       background-color: #2b2b2b;
                                       color: white;
                                       font-weight = bold;
                                       border : 2px sold white;
                                       padding : 5px;
                                       
                                       }
                                   """
           
            )
        self.button2.clicked.connect(self.K_Means)
        self.layout.addWidget(self.button2)
        booly = 0;


        self.button3 = QPushButton("Decision Tree")
        self.button3.setStyleSheet("""
                                   
                                   
                                   QPushButton {
                                       background-color: #2b2b2b;
                                       color: white;
                                       font-weight = bold;
                                       border : 2px sold white;
                                       padding : 5px;
                
                                      }
                                   """
           
            )
        self.button3.clicked.connect(self.DT)
        self.layout.addWidget(self.button3)
        
        self.central_widget.setLayout(self.layout)
        
    def show_message(self, message): 
        msg_box = QMessageBox(self)
        msg_box.setIcon(QMessageBox.Information)
        msg_box.setText(message)
        msg_box.setWindowTitle("Hows The Import Going? ")
        msg_box.setStyleSheet("background-color: #1b2b2b; color: white")
        msg_box.exec_()
        

    def load_image(self):
        """Open file dialog to load an image."""
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Open Image", "", "Image Files (*.png *.jpg *.jpeg *.bmp)", 
            options=options
        )
        if file_path:
            self.file_path = file_path
            pixmap = QPixmap(file_path)
            self.image_label.setPixmap(pixmap.scaled(400, 400)) # Scale image to fit
            self.show_message("Image Imported Successfully!")
        else:
            self.show_message("Couldn't Get The Image From This Destination")
    #here you'll have the image destination
        
    
    
    
    def clicked_thetopmessage(self, event):
        self.load_image()
#when you're pressing the exit button if you press the exit button the code should be shutted

    def ask_continue_or_quit(self, action_name):
                reply = QMessageBox.question(
                self,
                "continue or quit?",
                f"{action_name} executed successfully! \n\n Do you want to continue? or may I quit.",
                QMessageBox.Yes | QMessageBox.No,
                QMessageBox.Yes,
                )
            
                if reply == QMessageBox.Yes:
                    self.show_message("You chose to continue you may choose another algorithm\n press Ok and start again!")
                else :
                    QApplication.quit()
                
    def processing (self):
        
        original_image = cv2.imread(self.file_path)

        # Convert to RGB
        img_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    
        # Stage 1: CLAHE on RGB channels
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        r, g, b = cv2.split(img_rgb)
        r_clahe = clahe.apply(r)
        g_clahe = clahe.apply(g)
        b_clahe = clahe.apply(b)
        img_clahe = cv2.merge((r_clahe, g_clahe, b_clahe))
        output_path = 'image_clahe.jpg'  
        cv2.imwrite(output_path, img_clahe)

        # Stage 2: Thresholding (Disc and Cup)
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        gaussian_window_size = 5
        gray_gaussian = cv2.GaussianBlur(gray, (gaussian_window_size, gaussian_window_size), 0)
    
        # Calculate thresholds
        sigma_G = np.std(gray_gaussian)
        sigma_RI = np.std(img_rgb[:, :, 0])  # Red channel
        sigma_GI = np.std(img_rgb[:, :, 1])  # Green channel
        mu_GI = np.mean(img_rgb[:, :, 1])   # Green channel mean
        T1 = (0.5 * gaussian_window_size) - (2 * sigma_G) - sigma_RI
        T2 = (0.5 * gaussian_window_size) + (2 * sigma_G) + (2 * sigma_GI) + mu_GI
    
        # Apply thresholds
        _, thresh_disc = cv2.threshold(gray, abs(T1), 255, cv2.THRESH_BINARY)
        _, thresh_cup = cv2.threshold(gray, T2, 255, cv2.THRESH_BINARY)

        # Apply morphological opening and closing
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (13, 13))
        morph_open_disc = cv2.morphologyEx(thresh_disc, cv2.MORPH_OPEN, kernel)
        morph_close_disc = cv2.morphologyEx(morph_open_disc, cv2.MORPH_CLOSE, kernel)

        morph_open_cup = cv2.morphologyEx(thresh_cup, cv2.MORPH_OPEN, kernel)
        morph_close_cup = cv2.morphologyEx(morph_open_cup, cv2.MORPH_CLOSE, kernel)

        # Find contours
        contours_disc, _ = cv2.findContours(morph_close_disc, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours_cup, _ = cv2.findContours(morph_close_cup, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Apply convex hull to the contours
        hull_disc = [cv2.convexHull(cnt) for cnt in contours_disc]
        hull_cup = [cv2.convexHull(cnt) for cnt in contours_cup]

        # Draw contours on a blank image
        blank_disc = np.zeros_like(morph_close_disc)
        blank_cup = np.zeros_like(morph_close_cup)
        cv2.drawContours(blank_disc, hull_disc, -1, (255), -1)
        cv2.drawContours(blank_cup, hull_cup, -1, (255), -1)


        # Find largest contour (assuming the largest contour corresponds to the region of interest)
        largest_disc = max(hull_disc, key=cv2.contourArea)
        largest_cup = max(hull_cup, key=cv2.contourArea)

        # Fit ellipse to the largest contours
        ellipse_disc = cv2.fitEllipse(largest_disc)
        ellipse_cup = cv2.fitEllipse(largest_cup)

        # Draw ellipses on a blank image
        result_image = np.zeros_like(original_image)
        cv2.ellipse(result_image, ellipse_disc, (255, 0, 0), 2)  # Disc in red
        cv2.ellipse(result_image, ellipse_cup, (0, 255, 0), 2)  # Cup in green

        # Calculate major axes
        major_axis_disc = max(ellipse_disc[1])
        major_axis_cup = max(ellipse_cup[1])

        # Compute CDR
        cdr = major_axis_cup / major_axis_disc
        print(f"Cup-to-Disc Ratio (CDR): {cdr}")

        output_path3 = 'blank_disc.jpg'  
        cv2.imwrite(output_path3, blank_disc)
        output_path4 = 'blank_cup.jpg'  
        cv2.imwrite(output_path4, blank_cup)
        output_path5 = 'result_image.jpg'  
        cv2.imwrite(output_path5, result_image)

        self.show_processed_images("result_image.jpg", "blank_disc.jpg", "blank_cup.jpg")
        self.show_message(f"The calculated value of CDR is: {cdr}")
        
        
        
    def show_processed_images(self, processed_image1, processed_image2, 
                                          processed_image3 ) :
        self.window = QWidget()
        self.window.setWindowTitle("Processed Images")
        self.window.setGeometry(200, 100, 1200, 400)
        
        
        image_layout = QHBoxLayout()
        
        image_label0 = QLabel("Preprocess1")
        image_label0.setStyleSheet("""
                                       background-color: #1b2b2b; 
                                       color : white;
                                       border: 2px solid white; 
                                       """
            ) 
        image_label0.setAlignment(Qt.AlignCenter) 
        
        image_label1 = QLabel("Preprocess2")
        image_label1.setStyleSheet("""
                                       background-color: #1b2b2b; 
                                       color : white;
                                       border: 2px solid white; 
                                       """
            ) 
        image_label1.setAlignment(Qt.AlignCenter) 
        

        image_label2 = QLabel("Preprocess3")
        image_label2.setStyleSheet("""
                                       background-color: #1b2b2b; 
                                       color : white;
                                       border: 2px solid white; 
                                       """
            ) 
        image_label2.setAlignment(Qt.AlignCenter) 
        
        
        
        image_layout.addWidget(image_label0)
        image_layout.addWidget(image_label1)
        image_layout.addWidget(image_label2)
        
        pixmap = QPixmap(processed_image1)
        image_label0.setPixmap(pixmap.scaled(400, 400))
        
        pixmap = QPixmap(processed_image2)
        image_label1.setPixmap(pixmap.scaled(400, 400))
        
        pixmap = QPixmap(processed_image3)
        image_label2.setPixmap(pixmap.scaled(400, 400))
        
        
        
        
        self.window.setLayout(image_layout)
        self.window.show()
        
        
        
    """def convert_cv_to_pixmap(self, cv_image, is_grayscale = False) :
        
        cv2.imshow("hi", cv_image)
        
        a = QImage(cv_image.data, cv_image.shape[1], cv_image.shape[0], QImage.Format_RGB888).rgbSwapped()
        
        return QPixmap.fromImage(a)
        
        
        if is_grayscale :
            height, width, channel = cv_image.shape
            bytes_per_line = width
            q_image = QImage(cv_image.data, width, height, bytes_per_line,
                             QImage.Format_Grayscale8)
        
        else:
            rgb_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
            height, width, channel = rgb_image.shape
            bytes_per_line = channel * width
            q_image = QImage(rgb_image.data, width, height, bytes_per_line,
                             QImage.Format_Grayscale8
                                 )
        
        return QPixmap.fromImage(q_image)
    
        """

    def SVM(self):
        #add the first module here
        # Load the CNN model
        self.model = load_model('my_cnn_model.h5')
        # Load the SVM model
        with open('my_svm_model.pkl', 'rb') as file:
             svm_model = pickle.load(file)
           
        # Preprocess the image and predict using the CNN model
        image = cv2.imread(self.file_path)
        image = cv2.resize(image, (224, 224)) # Adjust size as needed
        image = image / 255.0  # Normalize the image
        image = numpy.expand_dims(image, axis=0)  # Add batch dimension

        cnn_prediction = self.model.predict(image)
           
        # Concatenate CDR value to the CNN output
        concatenated_output = numpy.append(cnn_prediction, [0.4])  # Example number to concatenate
           
        # Predict using the SVM model
        svm_prediction = svm_model.predict([concatenated_output])
#         print(svm_prediction[0])
           
        #define the boolean with name booly
        booly = svm_prediction[0]
        if booly == '1' :
            self.show_message("Unfortunately  Your Patient Has Glaucoma.")

        else : 
            self.show_message("Your Patient is Healthy.")
        
        self.ask_continue_or_quit("SVM")

    def K_Means(self):
        # Load the CNN model
        self.model = load_model('cnn_model.h5')
        
        # Load the KMeans model
        with open('kmeans_model.pkl', 'rb') as file:
            kmeans_model = pickle.load(file)
        
        # Load the saved scaler and PCA models
        with open('scaler.pkl', 'rb') as file:
            scaler = pickle.load(file)
        
        with open('pca.pkl', 'rb') as file:
            pca = pickle.load(file)
        
        # Preprocess the image and predict using the CNN model
        image = cv2.imread(self.file_path)
        image = cv2.resize(image, (224, 224))  # Resize image
        image = image / 255.0  # Normalize
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        
        cnn_prediction = self.model.predict(image)
        
        # Concatenate CDR value to the CNN output
        cdr_value = 0.4 # Example CDR value; replace with calculated CDR
        concatenated_output = np.append(cnn_prediction.flatten(), [cdr_value])
        concatenated_output = concatenated_output.reshape(1, -1)  # Reshape to 2D array

        
        # Normalize and apply PCA to match KMeans input
        concatenated_output_scaled = scaler.transform(concatenated_output)
        concatenated_output_pca = pca.transform(concatenated_output_scaled)
        
        # Predict using the KMeans model
        kmeans_prediction = kmeans_model.predict(concatenated_output_pca)
        
        # Show result
        booly = kmeans_prediction[0]
        if booly == '1':
            self.show_message("Unfortunately, your patient has Glaucoma.")
        else:
            self.show_message("Your patient is healthy.")
        
        self.ask_continue_or_quit("K-Means")

    def DT(self):
        #add the third module here
        # Load the CNN model
        self.model = load_model('my_cnn_model.h5')
        # Load the DT model
        with open('my_dt_model.pkl', 'rb') as file:
             dt_model = pickle.load(file)
        
        # Preprocess the image and predict using the CNN model
        image = cv2.imread(self.file_path)
        image = cv2.resize(image, (224, 224)) # Adjust size as needed
        image = image / 255.0  # Normalize the image
        image = numpy.expand_dims(image, axis=0)  # Add batch dimension

        cnn_prediction = self.model.predict(image)
           
         # Concatenate CDR value to the CNN output
        concatenated_output = numpy.append(cnn_prediction, [0.4])  # Example number to concatenate
           
        # Predict using the DT model
        dt_prediction = dt_model.predict([concatenated_output])
           
        #define the boolean with name booly
        booly = dt_prediction[0]
        if booly == '1' :
            self.show_message("Unfortunately  Your Patient Has Glaucoma.")

        else : 
            self.show_message("Your Patient is Healthy.")
        
        self.ask_continue_or_quit("DT")
            

   

    def closeEvent(self, event):
        reply = QMessageBox.question(self, 
                                     "Exit Application?", 
                                     "Are you sure you want to exit?" ,
                                     QMessageBox.No | QMessageBox.Yes,
                                     QMessageBox.No
                                     )
                                     
        if reply == QMessageBox.Yes :
            event.accept()
            QApplication.quit()
        else:
            event.ignore()
    

if __name__ == "__main__":
    
    app = QApplication(sys.argv)
    window = GetImageAndAddModule()
    window.show()
    sys.exit(app.exec_())

Cup-to-Disc Ratio (CDR): 0.7745685935830088


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 948ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


SystemExit: 0

C:\Users\My\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
